### <span style="color:#0b486b">Import Libraries</span>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

### <span style="color:#0b486b">Load Data</span>

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full_img, y_train_full), (X_test_img, y_test) = fashion_mnist.load_data()

In [ ]:
num_train = X_train_full_img.shape[0]
num_test = X_test_img.shape[0]
X_train_full = X_train_full_img.reshape(num_train,-1)
X_test = X_test_img.reshape(num_test,-1)
print(X_train_full.shape, y_train_full.shape)
print(X_test.shape, y_test.shape)

### <span style="color:#0b486b">Split Data</span>

In [ ]:
import math
N = X_train_full.shape[0]
i = math.floor(0.9*N)
X_train, y_train = X_train_full[0:i],y_train_full[0:i]
X_valid, y_valid = X_train_full[i:N],y_train_full[i:N]
X_train, X_valid, X_test = X_train/255.0, X_valid/255.0, X_test/255.0

### <span style="color:#0b486b">Visualize Data</span>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(10,10))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_full_img[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()


### <span style="color:#0b486b">Build Simple NN Model</span>

In [ ]:
tf.compat.v1.reset_default_graph()
tf.random.set_seed(1)
model = tf.keras.Sequential([
                    tf.keras.layers.Input(shape = 784),
                    tf.keras.layers.Dense(20, activation='relu'),
                    tf.keras.layers.Dense(25, activation= 'relu'),
                    tf.keras.layers.Dense(10, activation= 'softmax')])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=20)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print("The accuracy of the trained model on the testing set is",test_acc)

### <span style="color:#0b486b">Apply Hypter-tuning technique Grid Search to build better model</span>

In [ ]:
def createModel(nodeValue,activationValue):
    tf.compat.v1.reset_default_graph()
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = 784),
    tf.keras.layers.Dense(nodeValue, activation=activationValue),
    tf.keras.layers.Dense(25, activation= activationValue),
    tf.keras.layers.Dense(10, activation = "softmax")
    ])
    return model


In [ ]:
nodes = [10,20,30,40]
activation = ['sigmoid','tanh','relu']
test_acc_storage = []
best_test_acc = 0
best_nodes = nodes[0]
best_activation = activation[0]
tf.random.set_seed(1)

for i in range(len(nodes)):
    for j in range(len(activation)):
        
        model = createModel(nodes[i],activation[j])
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        model.fit(X_valid, y_valid,validation_data=(X_valid, y_valid), epochs=20, verbose = 0)
        test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)
        test_acc_storage.append(test_acc)
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_nodes = nodes[i]
            best_activation = activation[j]
            
print("The best number of nodes is:",best_nodes)
print("The best activation function is:",best_activation)
print("The test accuracy is:",best_test_acc)

### <span style="color:#0b486b">Experiment with temperature based model</span>

In [ ]:
fi = [0.1,0.5,0.8]
tf.random.set_seed(1)
modelStorage = []
for i in range(len(fi)):
    tf.compat.v1.reset_default_graph()
    X = tf.keras.layers.Input(shape = 784) #declare input layer
    h = tf.keras.layers.Dense(units=20, activation= 'relu')(X)
    h = tf.keras.layers.Dense(units=25, activation= 'relu')(h)
    h = h/fi[i]
    h = tf.keras.layers.Dense(units=10, activation= 'softmax')(h)
    model = tf.keras.Model(inputs= X, outputs=h)
    modelStorage.append(model)

temp_model = modelStorage[0] #choose which temperature you want
temp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
temp_model.fit(x= X_train, y= y_train, batch_size= 64, epochs= 20, validation_data = (X_valid, y_valid))